# Fertility rates by age of parent


The file begins by computing a counterfactual end-of-period population consistent with the population prevailing at the end of 1899, mortality rates, net migration, and live births created in the earlier files. The counterfactual population is needed to know how many adults of a given age are present at the end of each period. The counterfactual population can be compared to the actual population from the Census Bureau to ensure that the calculations are consisent with actual population dynamics. The counterfactual and actual populations need not be identical period by period, however, because of the moderate amount of smoothing that we introduce in light of our focus on low-frequency dynamics. 

The file then computes the end-of-period fertility rates by age of the representative adult. This information is used when solving a version of the model with backward-looking expectations. The fertility rates by age of parent series are measured at the end of the period as ratios featuring, on the numerator, the number of surviving babies born of parents of a given age, and, on the denominator, the number of survivng adults of that age:

$$Fertitlity\_rate\_adult_{age,t} = \frac{Population_{0,t} \times Share\_baby\_adult_{age,t}}{Population_{age,t}}.$$

The variable $Share\_baby\_adult_{age,t}$ is computed by allocating babies to their parents taking into account the age of the mother and the average age difference between men and women at first marriage.

Our fertility rate statistics apply to representative adults whereas official fertility rates typically apply to mothers. Moreover, our fertility rate series contain an adjustment for the survival of parents, which need not be present in official fertility rate statistics. Thus, the two fertility rate series need not be comparable.

### Counterfactual population (annual)

In [ ]:
# Read mortality rates, population end of 1899, live births, and net migration
using DelimitedFiles
births_Y = readdlm("CleanData/births_Y.csv", ',');
netmigration_Y = readdlm("CleanData/netmigration_Y.csv", ',');
Γ_AGEy_PERy = readdlm("CleanData/interp_death_rate_1900_2220_Y.csv", ',');
population_1899_Y = readdlm("CleanData/population_1899_Y.csv", ',');

# Iteratively construct population (first period)
Population_counterfactual_Y = zeros(size(netmigration_Y));
Population_counterfactual_Y[1,1] = (births_Y[1,1] + netmigration_Y[1,1])*(1-Γ_AGEy_PERy[1,1])
Population_counterfactual_Y[2:end,1] = (population_1899_Y[1:end-1,1]+netmigration_Y[2:end,1]).*(1.0.-Γ_AGEy_PERy[2:end,1])

# Iteratively construct population (subsequent periods)
for t=2:size(netmigration_Y,2)
    Population_counterfactual_Y[1,t] = (births_Y[t,1] + netmigration_Y[1,t])*(1-Γ_AGEy_PERy[1,t])
    Population_counterfactual_Y[2:end,t] = (Population_counterfactual_Y[1:end-1,t-1]+netmigration_Y[2:end,t]).*(1.0.-Γ_AGEy_PERy[2:end,t])
end

# Writing population
writedlm("CleanData/population_counterfactual_Y.csv", Population_counterfactual_Y, header = false, ',')

### Counterfactual population (quarterly)

In [ ]:
# Read mortality rates, population end of 1899, live births, and net migration
births_Q = readdlm("CleanData/births_Q_annualized.csv", ',');
netmigration_Q = readdlm("CleanData/netmigration_Q.csv", ',');
Γ_AGEq_PERq = readdlm("CleanData/interp_death_rate_1900_2220_Q.csv", ',');
population_1899_Q = readdlm("CleanData/population_1899_Q.csv", ',');

# Iteratively construct population (first period)
Population_counterfactual_Q = zeros(size(netmigration_Q));
Population_counterfactual_Q[1,1] = (births_Q[1,1]/4 + netmigration_Q[1,1])*((1.0.-Γ_AGEq_PERq[1,1]).^0.25)
Population_counterfactual_Q[2:end,1] = (population_1899_Q[1:end-1,1]+netmigration_Q[2:end,1]).*((1.0.-Γ_AGEq_PERq[2:end,1]).^0.25)

# Iteratively construct population (subsequent periods)
for t=2:size(netmigration_Q,2)
    Population_counterfactual_Q[1,t] = (births_Q[t,1]/4 + netmigration_Q[1,t])*((1-Γ_AGEq_PERq[1,t])^0.25)
    Population_counterfactual_Q[2:end,t] = (Population_counterfactual_Q[1:end-1,t-1]+netmigration_Q[2:end,t]).*((1.0.-Γ_AGEq_PERq[2:end,t]).^0.25)
end

# Writing population
writedlm("CleanData/population_counterfactual_Q.csv", Population_counterfactual_Q, header = false, ',')

### Fertility rates (annual)

In [ ]:
# Declaring matrices of results
Parent_fertility_Y = zeros(size(netmigration_Y));
Parent_newborns_Y = zeros(size(netmigration_Y));

# Reading share of births by age of mother
share_births_mothers_Y = readdlm("CleanData/share_births_mothers_Y.csv", ',');
MarriageAgeDiff_Y = readdlm("CleanData/marriageAgeDiff_Y.csv", ',');

# Allocating births by age of parent (with smoothing of fractional age difference across age bins)
for tt=1:size(netmigration_Y,2)
    Parent_newborns_Y[Array{Int64,1}(collect(15:50).+floor(MarriageAgeDiff_Y[tt,1]/2)),tt] = (1-rem(MarriageAgeDiff_Y[tt,1]/2,1))*(births_Y[tt,1]+netmigration_Y[1,tt])*share_births_mothers_Y[:,tt]
    Parent_newborns_Y[Array{Int64,1}(collect(15:50).+ceil( MarriageAgeDiff_Y[tt,1]/2)),tt] = Parent_newborns_Y[Array{Int64,1}(collect(15:50).+ceil(MarriageAgeDiff_Y[tt,1]/2)),tt] + rem(MarriageAgeDiff_Y[tt,1]/2,1)*(births_Y[tt]+netmigration_Y[1,tt])*share_births_mothers_Y[:,tt]
    Parent_newborns_Y[:,tt] = Parent_newborns_Y[:,tt]*(1.0.-Γ_AGEy_PERy[1,tt])
    Parent_fertility_Y[:,tt] = Parent_newborns_Y[:,tt]./Population_counterfactual_Y[:,tt]
end

# Writing fertility rates
writedlm("CleanData/parent_fertility_Y.csv", Parent_fertility_Y, header = false, ',')

### Fertility rates (quarterly)

In [ ]:
# Declaring matrices of results
Parent_fertility_Q = zeros(size(netmigration_Q));
Parent_newborns_Q = zeros(size(netmigration_Q));

# Reading share of births by age of mother
share_births_mothers_Q = readdlm("CleanData/share_births_mothers_Q.csv", ',');
MarriageAgeDiff_Q = readdlm("CleanData/marriageAgeDiff_Q.csv", ',')

# Allocating births by age of parent (with smoothing of fractional age difference across age bins)
for qq=1:size(netmigration_Q,2)
    Parent_newborns_Q[Array{Int64,1}(collect(57:200).+floor(MarriageAgeDiff_Q[qq,1]*2.0)),qq] = (0.25-rem(MarriageAgeDiff_Q[qq,1],0.25))/0.25*(births_Q[qq,1]/4+netmigration_Q[1,qq])*share_births_mothers_Q[:,qq]
    Parent_newborns_Q[Array{Int64,1}(collect(57:200).+ ceil(MarriageAgeDiff_Q[qq,1]*2.0)),qq] = Parent_newborns_Q[Array{Int64,1}(collect(57:200).+ceil(MarriageAgeDiff_Q[qq,1]*2.0)),qq] + rem(MarriageAgeDiff_Q[qq,1],0.25)/0.25*(births_Q[qq]/4+netmigration_Q[1,qq])*share_births_mothers_Q[:,qq]
    Parent_newborns_Q[:,qq] = Parent_newborns_Q[:,qq].*(1.0.-Γ_AGEq_PERq[1,qq]).^(0.25)
    Parent_fertility_Q[:,qq] = Parent_newborns_Q[:,qq]./Population_counterfactual_Q[:,qq]
end

# Writing fertility rates
writedlm("CleanData/parent_fertility_Q.csv", Parent_fertility_Q, header = false, ',')

In [ ]:
# Changing permissions
run(`chmod 664 CleanData/population_counterfactual_Y.csv`);
run(`chmod 664 CleanData/population_counterfactual_Q.csv`);
run(`chmod 664 CleanData/parent_fertility_Y.csv`);
run(`chmod 664 CleanData/parent_fertility_Q.csv`);